<a href="https://colab.research.google.com/github/fredfurst/som/blob/master/SOM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Taking from https://github.com/lyes-khacef/GPU-SOM

Now mixing it up with data from a notebook in the ChaosDoc folder

## 1. Prep

### GPUtil

In [1]:
!pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7394 sha256=f59cde4206037c4333c3d3e672bb5f573bb1de3d8aae7b8ee4ade2cac06dbef8
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil


### From imports.py

In [2]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import timeit

# import packages
import os, sys, humanize, psutil, GPUtil

# diable gpu
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"

# disable tensorflow logs
"""
0 = all messages are logged (default behavior)
1 = INFO messages are not printed
2 = INFO and WARNING messages are not printed
3 = INFO, WARNING, and ERROR messages are not printed
"""
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

# import tensorflow
import tensorflow as tf
print("Tensorflow version = ", tf.__version__)
if tf.executing_eagerly():
    print("Eager execution!")
#tf.debugging.set_log_device_placement(True)

Tensorflow version =  2.15.0
Eager execution!


### From gpu_check.py

Check for GPUs...

In [3]:
# imports
#from imports import *

# define function
def gpu_report():
    print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
    GPUs = GPUtil.getGPUs()
    print(GPUs)
    for i, gpu in enumerate(GPUs):
        print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))

# GPU memory check
gpu_report()

CPU RAM Free: 12.4 GB
GPU 0 ... Mem Free: 15101MB / 15360MB | Utilization   0%


## 2. Uploading file

In [4]:
from google.colab import files
uploaded_file = files.upload()

KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
import pandas as pd
import io

In [ ]:
uploaded_file.keys()

In [ ]:
df1 = pd.read_fwf(
    io.BytesIO(uploaded_file['e_composite_im7_8552_OHT_b-OHT-Hard-J3-ExpRed_S11.csv']),
    header=[1,2,3],
    skip_blank_lines=True)

In [ ]:
df1.tail()

### Get Column names For further vizes...

In [ ]:
df1.head()

In [ ]:
npdf1 = np.array(df1[:400],dtype=float).transpose()

In [ ]:
npdf1.shape

# 3. Boilerplate

### From data_load.py

In [ ]:
# imports
#from imports import *
from tensorflow.keras.datasets import mnist

# fuction to load data from file
def load_data(name):
    data = np.load(name, allow_pickle=True)
    print("Data.shape = ", data.shape)
    return data

def get_dataset(train_data, label_data, test_data):
    # importing dataset
    (x_train_all, index_train_all), (x_test_all, index_test_all) = mnist.load_data()
    x_train_all = x_train_all.astype('float32') / 255.
    x_test_all = x_test_all.astype('float32') / 255.
    x_train_all = x_train_all.reshape((60000, 784))
    x_test_all = x_test_all.reshape((10000, 784))

    # constructing dataset
    x_train = np.copy(x_train_all[:train_data,:])
    index_train = np.copy(index_train_all[:train_data])
    x_label = np.copy(x_train_all[:label_data,:])
    index_label = np.copy(index_train_all[:label_data])
    x_test = np.copy(x_test_all[:test_data,:])
    index_test = np.copy(index_test_all[:test_data])

    return x_train, index_train, x_label, index_label, x_test, index_test, label_data

## Models

### From file tf_ksom.py

In [ ]:
class KSOM_3D():
    def __init__(self, m, n, o, dim):
        self.m = m
        self.n = n
        self.o = o
        self.dim = dim
        self.map_wgt =  tf.Variable(
                            tf.random.uniform(
                                shape = [m*n*o, dim],
                                minval = 0.0,
                                maxval = 1.0,
                                dtype = tf.float32,
                                seed = 23
                            )
                        )
        self.map_loc =  tf.constant(
                            np.array(
                                list(self.neuron_locs(m, n, o))
                            )
                        )

    def neuron_locs(self, m, n, o):
        # nested iterations over both dimensions to yield one by one the 2-d locations of the individual neurons in the SOM
        for i in range(m):
            for j in range(n):
                for k in range(o):
                    yield np.array([i,j,k], dtype=np.float32)

    def compute_winner(self, sample):
            self.sample = sample

            # compute the squared euclidean distance between the input and the neurons
            self.squared_distance = tf.reduce_sum(
                                        tf.square(
                                            tf.subtract(
                                                self.map_wgt, # [m*n*o, dim]
                                                tf.expand_dims(
                                                    self.sample, # [dim] -> [1, dim]
                                                    axis=0
                                                )
                                            )
                                        ),
                                        axis=1
                                    )

            # find the bmu's index
            self.bmu_idx =  tf.argmin(
                                    input=self.squared_distance,
                                    axis=0
                                )

            # extract the bmu's 2-d location
            self.bmu_loc =  tf.gather(
                                self.map_loc,
                                self.bmu_idx
                            )

    def update_network(self, epsilon, eta):
        # compute the squared manhattan distance between the bmu and the neurons
        self.bmu_distance_squares = tf.reduce_sum(
                                        tf.square(
                                            tf.subtract(
                                                self.map_loc, # [m*n*o, 2]
                                                tf.expand_dims(
                                                    self.bmu_loc, # [2] -> [1, 2]
                                                    axis=0
                                                )
                                            )
                                        ),
                                        axis=1
                                    )

        # compute the neighborhood function
        self.neighbourhood_func = tf.exp(
                                      tf.negative(
                                          tf.math.divide(
                                              self.bmu_distance_squares,
                                              tf.multiply(
                                                  tf.square(
                                                      eta,
                                                  ),
                                                  2.0
                                              )
                                          )
                                      )
                                  )

        # compute the overall learning of each neuron
        self.learning = tf.multiply(
                            self.neighbourhood_func,
                            epsilon
                        )

        # compute the difference between the neurons weights and the input
        self.delta_wgt =  tf.subtract(
                              tf.expand_dims(
                                  self.sample, # [dim] -> [1, dim]
                                  axis=0
                              ),
                              self.map_wgt, # [m*n*o, dim]
                          )

        # compute the weights update according to the learning and delta_wgt and update the weights
        tf.compat.v1.assign_add(
            self.map_wgt,
            tf.multiply(
                tf.expand_dims(
                    self.learning, # [m*n*o] -> [m*n*o, 1]
                    axis=-1
                ),
                self.delta_wgt # [m*n*o, dim]
            )
        )

    def get_weights(self):
        return self.map_wgt

    @tf.function
    def train(self, nbr_epochs, epsilon_i, epsilon_f, eta_i, eta_f, x_train, x_label, index_label, x_test, index_test):
        with tf.device('/device:gpu:0'):
            for epoch in tf.range(nbr_epochs):
                tf.print("---------- Epoch", epoch + 1, "----------")

                # update the learning rate epsilon
                epsilon_t =  tf.multiply(
                                    epsilon_i,
                                    tf.pow(
                                        tf.math.divide(
                                            epsilon_f,
                                            epsilon_i
                                        ),
                                        tf.cast(
                                            tf.math.divide(
                                                epoch,
                                                nbr_epochs - 1
                                            ),
                                            dtype=tf.float32
                                        )
                                    )
                                )

                # update the gaussian neighborhood witdh eta
                eta_t =  tf.multiply(
                                  eta_i,
                                  tf.pow(
                                      tf.math.divide(
                                          eta_f,
                                          eta_i
                                      ),
                                      tf.cast(
                                          tf.math.divide(
                                              epoch,
                                              nbr_epochs - 1
                                          ),
                                          dtype=tf.float32
                                      )
                                  )
                              )

                # shuffle the training dataset
                tf.random.shuffle(x_train)

                # bmu computing and network update for each sample
                for x_trn in x_train:
                    sample = tf.cast(x_trn, dtype=tf.float32)
                    self.compute_winner(sample)
                    self.update_network(epsilon_t, eta_t)

In [ ]:
# imports
#from imports import *

class KSOM():
    def __init__(self, m, n, dim):
        self.m = m
        self.n = n
        self.dim = dim
        self.map_wgt =  tf.Variable(
                            tf.random.uniform(
                                shape = [m*n, dim],
                                minval = 0.0,
                                maxval = 1.0,
                                dtype = tf.float32,
                                seed = 23
                            )
                        )
        self.map_loc =  tf.constant(
                            np.array(
                                list(self.neuron_locs(m, n))
                            )
                        )

    def neuron_locs(self, m, n):
        # nested iterations over both dimensions to yield one by one the 2-d locations of the individual neurons in the SOM
        for i in range(m):
            for j in range(n):
                yield np.array([i,j], dtype=np.float32)

    def compute_winner(self, sample):
            self.sample = sample

            # compute the squared euclidean distance between the input and the neurons
            self.squared_distance = tf.reduce_sum(
                                        tf.square(
                                            tf.subtract(
                                                self.map_wgt, # [m*n, dim]
                                                tf.expand_dims(
                                                    self.sample, # [dim] -> [1, dim]
                                                    axis=0
                                                )
                                            )
                                        ),
                                        axis=1
                                    )

            # find the bmu's index
            self.bmu_idx =  tf.argmin(
                                    input=self.squared_distance,
                                    axis=0
                                )

            # extract the bmu's 2-d location
            self.bmu_loc =  tf.gather(
                                self.map_loc,
                                self.bmu_idx
                            )

    def update_network(self, epsilon, eta):
        # compute the squared manhattan distance between the bmu and the neurons
        self.bmu_distance_squares = tf.reduce_sum(
                                        tf.square(
                                            tf.subtract(
                                                self.map_loc, # [m*n, 2]
                                                tf.expand_dims(
                                                    self.bmu_loc, # [2] -> [1, 2]
                                                    axis=0
                                                )
                                            )
                                        ),
                                        axis=1
                                    )

        # compute the neighborhood function
        self.neighbourhood_func = tf.exp(
                                      tf.negative(
                                          tf.math.divide(
                                              self.bmu_distance_squares,
                                              tf.multiply(
                                                  tf.square(
                                                      eta,
                                                  ),
                                                  2.0
                                              )
                                          )
                                      )
                                  )

        # compute the overall learning of each neuron
        self.learning = tf.multiply(
                            self.neighbourhood_func,
                            epsilon
                        )

        # compute the difference between the neurons weights and the input
        self.delta_wgt =  tf.subtract(
                              tf.expand_dims(
                                  self.sample, # [dim] -> [1, dim]
                                  axis=0
                              ),
                              self.map_wgt, # [m*n, dim]
                          )

        # compute the weights update according to the learning and delta_wgt and update the weights
        tf.compat.v1.assign_add(
            self.map_wgt,
            tf.multiply(
                tf.expand_dims(
                    self.learning, # [m*n] -> [m*n, 1]
                    axis=-1
                ),
                self.delta_wgt # [m*n, dim]
            )
        )

    def get_weights(self):
        return self.map_wgt

    @tf.function
    def train(self, nbr_epochs, epsilon_i, epsilon_f, eta_i, eta_f, x_train, x_label, index_label, x_test, index_test):
        with tf.device('/device:gpu:0'):
            for epoch in tf.range(nbr_epochs):
                tf.print("---------- Epoch", epoch + 1, "----------")

                # update the learning rate epsilon
                epsilon_t =  tf.multiply(
                                    epsilon_i,
                                    tf.pow(
                                        tf.math.divide(
                                            epsilon_f,
                                            epsilon_i
                                        ),
                                        tf.cast(
                                            tf.math.divide(
                                                epoch,
                                                nbr_epochs - 1
                                            ),
                                            dtype=tf.float32
                                        )
                                    )
                                )

                # update the gaussian neighborhood witdh eta
                eta_t =  tf.multiply(
                                  eta_i,
                                  tf.pow(
                                      tf.math.divide(
                                          eta_f,
                                          eta_i
                                      ),
                                      tf.cast(
                                          tf.math.divide(
                                              epoch,
                                              nbr_epochs - 1
                                          ),
                                          dtype=tf.float32
                                      )
                                  )
                              )

                # shuffle the training dataset
                tf.random.shuffle(x_train)

                # bmu computing and network update for each sample
                for x_trn in x_train:
                    sample = tf.cast(x_trn, dtype=tf.float32)
                    self.compute_winner(sample)
                    self.update_network(epsilon_t, eta_t)

### From label.py

In [ ]:
# imports
#from imports import *
from test import *

def labeling(label_data, class_nbr, weights, x_label, index_label, sigma):
    dist_sum = np.zeros((len(weights), class_nbr))
    nbr_digits = np.zeros((class_nbr,))

    # accumulate the normalized gaussian distance for the labeling dataset
    for (x, y) in zip(x_label, index_label):
        nbr_digits[y] += 1
        dist_neuron = np.exp(-np.linalg.norm(x - weights, axis=1)/sigma)
        dist_bmu = np.max(dist_neuron)
        for i, distn in enumerate(dist_neuron):
            dist_sum[i][y] += distn/dist_bmu

    # normalize the activities on the number of samples per class
    for i, dists in enumerate(dist_sum):
        dist_sum[i] = dists/nbr_digits

    # assign the neurons labels
    neuron_label = np.argmax(dist_sum, axis=1)
    print("Neurons labels = ")
    print(neuron_label)

    return neuron_label

def unison_shuffle(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def run_labeling(weights, label_data, x_tr, index_tr, x_ts, index_ts):
    class_nbr = 10
    sigma_kernel = 1.0
    for i in range(10):
        x_tr, index_tr = unison_shuffle(x_tr, index_tr)
        x_lb = np.copy(x_tr[:label_data,:])
        index_lb = np.copy(index_tr[:label_data])

        # label the network
        neuron_label = labeling(label_data, class_nbr, weights, x_lb, index_lb, sigma_kernel)

        # test the network
        accuracy = test(class_nbr, weights, x_ts, index_ts, neuron_label, sigma_kernel)

### From test.py

In [ ]:
# from imports import *

def test(class_nbr, weights, x_test, index_test, neuron_label, sigma):
    neuron_index = np.zeros((len(x_test),), dtype=int)

    # calculate the BMUs for the test dataset
    for i, x in enumerate(x_test):
        dist_neuron = np.linalg.norm(x - weights, axis=1)
        neuron_index[i] = np.argmin(dist_neuron)

    # compare the BMUs labels and the samples labels
    accuracy = 0
    for p, t in zip(neuron_index, index_test):
        if neuron_label[p] == t:
            accuracy += 1
    accuracy = (float(accuracy)/len(x_test))*100
    print("SOM test accuracy = %.2f\n" % accuracy)

    return accuracy

# Adjustments

## From main.py

#### Later

##### Mapa 20 por 20 = 400 classes

In [ ]:
# -*- coding: utf-8 -*-

# ####################################################################################################
# GPU-based Self-Organizing-Map by Lyes Khacef.
# Reference: L. Khacef, V. Gripon, and B. Miramond, “GPU-based self-organizing-maps
# for post-labeled few-shot unsupervised learning”, in International Conference On
# Neural Information Processing (ICONIP), 2020.
# ####################################################################################################

# hyper-parameters
train_data = 60000
label_data = 600
test_data = 10000
input_dim = 400 #784
map_wth = 20 # 10
map_hgt = 20 # 10
map_dpt = 8
class_nbr = 10
nbr_epochs = 40 # 20
eps_i_list = [1.0]
eps_f_list = [0.01]
eta_i_list = [10.0]
eta_f_list = [0.01]
sigma_kernel = 1.0

# GPU name
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError("GPU device not found!")
print('Found GPU at: {}'.format(device_name))

##
## So, what if theres not test data
##
# load dataset
# x_train, index_train, x_label, index_label, x_test, index_test, label_data = get_dataset(train_data, label_data, test_data)
index_train, x_label, index_label, x_test, index_test, label_data = False, False, False, False, False, False
x_train = npdf1

##
## how the hell
##
# false run_som(false, false, false, eta_f):
def run_som(eps_i, eps_f, eta_i, eta_f):
    print("\nHyper-parameters:   # eps_i = %f   # eps_f = %f   # eta_i = %f   # eta_f = %f" % (eps_i, eps_f, eta_i, eta_f))
    # train the network
    som = KSOM_3D(map_wth, map_hgt, map_dpt, input_dim)
    start_time = timeit.default_timer()
    som.train(nbr_epochs, eps_i, eps_f, eta_i, eta_f, x_train, x_label, index_label, x_test, index_test)
    end_time = timeit.default_timer()
    print("\nSOM training time = ", end_time - start_time)
    weights = som.get_weights().numpy()

    # save the weights
    #np.save("weights/som_weights.npy", weights)

    ##
    ## Now you no label
    ##
    # label the network
    #neuron_label = labeling(label_data, class_nbr, weights, x_label, index_label, sigma_kernel)
    neuron_label = False

    ##
    ## Now you no accuracy
    # test the network
    #accuracy = test(class_nbr, weights, x_test, index_test, neuron_label, sigma_kernel)
    accuracy = False

    return weights, accuracy

# hyper-parameters grid search
hyper_param_list, accuracy_list = [], []
for eps_i in eps_i_list:
    for eps_f in eps_f_list:
        for eta_i in eta_i_list:
            for eta_f in eta_f_list:
                hyper_param_list.append([eps_i, eps_f, eta_i, eta_f])
                weights, accuracy = run_som(eps_i, eps_f, eta_i, eta_f)
                accuracy_list.append(accuracy)


In [ ]:
# GPU memory check
gpu_report()

In [ ]:
for k in range(map_dpt):
  # display neurons weights as mnist digits
  som_grid = plt.figure(figsize=(20, 20)) # width, height in inches
  for n in range(map_wth*map_hgt):
      ##
      ## Must make this into time series plot. Shouldnt be the thing
      ##
      #image = weights[n].reshape([28,28]) # x_train[num] is the 784 normalized pixel values
      sub = som_grid.add_subplot(map_wth, map_hgt, n + 1)
      sub.set_axis_off()
      clr = sub.plot(weights[n][k]) # imshow(image, cmap = plt.get_cmap("jet"), interpolation = "nearest")
      #plt.colorbar(clr)
  #plt.savefig("plots/som_weights.png")
  plt.show()

##### Mapa 10 por 10 = 100 classes

In [ ]:
# -*- coding: utf-8 -*-

# ####################################################################################################
# GPU-based Self-Organizing-Map by Lyes Khacef.
# Reference: L. Khacef, V. Gripon, and B. Miramond, “GPU-based self-organizing-maps
# for post-labeled few-shot unsupervised learning”, in International Conference On
# Neural Information Processing (ICONIP), 2020.
# ####################################################################################################

# imports
#from imports import *
#from gpu_check import *
#from data_load import *
#from tf_ksom import *
#from label import *
#from test import *

# hyper-parameters
train_data = 60000
label_data = 600
test_data = 10000
input_dim = 400 #784
map_wth = 10
map_hgt = 10
class_nbr = 10
nbr_epochs = 40 # 20
eps_i_list = [1.0]
eps_f_list = [0.01]
eta_i_list = [10.0]
eta_f_list = [0.01]
sigma_kernel = 1.0

# GPU name
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError("GPU device not found!")
print('Found GPU at: {}'.format(device_name))

##
## So, what if theres not test data
##
# load dataset
# x_train, index_train, x_label, index_label, x_test, index_test, label_data = get_dataset(train_data, label_data, test_data)
index_train, x_label, index_label, x_test, index_test, label_data = False, False, False, False, False, False
x_train = npdf1

##
## how the hell
##
# false run_som(false, false, false, eta_f):
def run_som(eps_i, eps_f, eta_i, eta_f):
    print("\nHyper-parameters:   # eps_i = %f   # eps_f = %f   # eta_i = %f   # eta_f = %f" % (eps_i, eps_f, eta_i, eta_f))
    # train the network
    som = KSOM(map_wth, map_hgt, input_dim)
    start_time = timeit.default_timer()
    som.train(nbr_epochs, eps_i, eps_f, eta_i, eta_f, x_train, x_label, index_label, x_test, index_test)
    end_time = timeit.default_timer()
    print("\nSOM training time = ", end_time - start_time)
    weights = som.get_weights().numpy()

    # save the weights
    #np.save("weights/som_weights.npy", weights)

    ##
    ## Now you no label
    ##
    # label the network
    #neuron_label = labeling(label_data, class_nbr, weights, x_label, index_label, sigma_kernel)
    neuron_label = False

    ##
    ## Now you no accuracy
    # test the network
    #accuracy = test(class_nbr, weights, x_test, index_test, neuron_label, sigma_kernel)
    accuracy = False

    return weights, accuracy

# hyper-parameters grid search
hyper_param_list, accuracy_list = [], []
for eps_i in eps_i_list:
    for eps_f in eps_f_list:
        for eta_i in eta_i_list:
            for eta_f in eta_f_list:
                hyper_param_list.append([eps_i, eps_f, eta_i, eta_f])
                weights, accuracy = run_som(eps_i, eps_f, eta_i, eta_f)
                accuracy_list.append(accuracy)


In [ ]:
## hyper parameters below
## label list below

# GPU memory check
gpu_report()

# display neurons weights as mnist digits
som_grid = plt.figure(figsize=(200, 200)) # width, height in inches
for n in range(map_wth*map_hgt):
    ##
    ## Must make this into time series plot. Shouldnt be the thing
    ##
    #image = weights[n].reshape([28,28]) # x_train[num] is the 784 normalized pixel values
    sub = som_grid.add_subplot(map_wth, map_hgt, n + 1)
    sub.set_axis_off()
    clr = sub.plot(weights[n]) # imshow(image, cmap = plt.get_cmap("jet"), interpolation = "nearest")
    #plt.colorbar(clr)
#plt.savefig("plots/som_weights.png")
plt.show()

##### Mapa 5 por 5 = 25 classes

In [ ]:
# -*- coding: utf-8 -*-

# ####################################################################################################
# GPU-based Self-Organizing-Map by Lyes Khacef.
# Reference: L. Khacef, V. Gripon, and B. Miramond, “GPU-based self-organizing-maps
# for post-labeled few-shot unsupervised learning”, in International Conference On
# Neural Information Processing (ICONIP), 2020.
# ####################################################################################################

# imports
#from imports import *
#from gpu_check import *
#from data_load import *
#from tf_ksom import *
#from label import *
#from test import *

# hyper-parameters
train_data = 60000
label_data = 600
test_data = 10000
input_dim = 400 #784
map_wth = 5
map_hgt = 5
class_nbr = 10
nbr_epochs = 40 # 20
eps_i_list = [1.0]
eps_f_list = [0.01]
eta_i_list = [10.0]
eta_f_list = [0.01]
sigma_kernel = 1.0

# GPU name
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError("GPU device not found!")
print('Found GPU at: {}'.format(device_name))

##
## So, what if theres not test data
##
# load dataset
# x_train, index_train, x_label, index_label, x_test, index_test, label_data = get_dataset(train_data, label_data, test_data)
index_train, x_label, index_label, x_test, index_test, label_data = False, False, False, False, False, False
x_train = npdf1

##
## how the hell
##
# false run_som(false, false, false, eta_f):
def run_som(eps_i, eps_f, eta_i, eta_f):
    print("\nHyper-parameters:   # eps_i = %f   # eps_f = %f   # eta_i = %f   # eta_f = %f" % (eps_i, eps_f, eta_i, eta_f))
    # train the network
    som = KSOM(map_wth, map_hgt, input_dim)
    start_time = timeit.default_timer()
    som.train(nbr_epochs, eps_i, eps_f, eta_i, eta_f, x_train, x_label, index_label, x_test, index_test)
    end_time = timeit.default_timer()
    print("\nSOM training time = ", end_time - start_time)
    weights = som.get_weights().numpy()

    # save the weights
    #np.save("weights/som_weights.npy", weights)

    ##
    ## Now you no label
    ##
    # label the network
    #neuron_label = labeling(label_data, class_nbr, weights, x_label, index_label, sigma_kernel)
    neuron_label = False

    ##
    ## Now you no accuracy
    # test the network
    #accuracy = test(class_nbr, weights, x_test, index_test, neuron_label, sigma_kernel)
    accuracy = False

    return weights, accuracy

# hyper-parameters grid search
hyper_param_list, accuracy_list = [], []
for eps_i in eps_i_list:
    for eps_f in eps_f_list:
        for eta_i in eta_i_list:
            for eta_f in eta_f_list:
                hyper_param_list.append([eps_i, eps_f, eta_i, eta_f])
                weights, accuracy = run_som(eps_i, eps_f, eta_i, eta_f)
                accuracy_list.append(accuracy)


In [ ]:
## hyper parameters below
## label list below

# GPU memory check
gpu_report()

# display neurons weights as mnist digits
som_grid = plt.figure(figsize=(200, 200)) # width, height in inches
for n in range(map_wth*map_hgt):
    ##
    ## Must make this into time series plot. Shouldnt be the thing
    ##
    #image = weights[n].reshape([28,28]) # x_train[num] is the 784 normalized pixel values
    sub = som_grid.add_subplot(map_wth, map_hgt, n + 1)
    sub.set_axis_off()
    clr = sub.plot(weights[n]) # imshow(image, cmap = plt.get_cmap("jet"), interpolation = "nearest")
    #plt.colorbar(clr)
#plt.savefig("plots/som_weights.png")
plt.show()

##### Mapa 4 por 4 = 16 classes

In [ ]:
# -*- coding: utf-8 -*-

# ####################################################################################################
# GPU-based Self-Organizing-Map by Lyes Khacef.
# Reference: L. Khacef, V. Gripon, and B. Miramond, “GPU-based self-organizing-maps
# for post-labeled few-shot unsupervised learning”, in International Conference On
# Neural Information Processing (ICONIP), 2020.
# ####################################################################################################

# imports
#from imports import *
#from gpu_check import *
#from data_load import *
#from tf_ksom import *
#from label import *
#from test import *

# hyper-parameters
train_data = 60000
label_data = 600
test_data = 10000
input_dim = 400 #784
map_wth = 4
map_hgt = 4
class_nbr = 10
nbr_epochs = 40 # 20
eps_i_list = [1.0]
eps_f_list = [0.01]
eta_i_list = [10.0]
eta_f_list = [0.01]
sigma_kernel = 1.0

# GPU name
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError("GPU device not found!")
print('Found GPU at: {}'.format(device_name))

##
## So, what if theres not test data
##
# load dataset
# x_train, index_train, x_label, index_label, x_test, index_test, label_data = get_dataset(train_data, label_data, test_data)
index_train, x_label, index_label, x_test, index_test, label_data = False, False, False, False, False, False
x_train = npdf1

##
## how the hell
##
# false run_som(false, false, false, eta_f):
def run_som(eps_i, eps_f, eta_i, eta_f):
    print("\nHyper-parameters:   # eps_i = %f   # eps_f = %f   # eta_i = %f   # eta_f = %f" % (eps_i, eps_f, eta_i, eta_f))
    # train the network
    som = KSOM(map_wth, map_hgt, input_dim)
    start_time = timeit.default_timer()
    som.train(nbr_epochs, eps_i, eps_f, eta_i, eta_f, x_train, x_label, index_label, x_test, index_test)
    end_time = timeit.default_timer()
    print("\nSOM training time = ", end_time - start_time)
    weights = som.get_weights().numpy()

    # save the weights
    #np.save("weights/som_weights.npy", weights)

    ##
    ## Now you no label
    ##
    # label the network
    #neuron_label = labeling(label_data, class_nbr, weights, x_label, index_label, sigma_kernel)
    neuron_label = False

    ##
    ## Now you no accuracy
    # test the network
    #accuracy = test(class_nbr, weights, x_test, index_test, neuron_label, sigma_kernel)
    accuracy = False

    return weights, accuracy

# hyper-parameters grid search
hyper_param_list, accuracy_list = [], []
for eps_i in eps_i_list:
    for eps_f in eps_f_list:
        for eta_i in eta_i_list:
            for eta_f in eta_f_list:
                hyper_param_list.append([eps_i, eps_f, eta_i, eta_f])
                weights, accuracy = run_som(eps_i, eps_f, eta_i, eta_f)
                accuracy_list.append(accuracy)


In [ ]:
## hyper parameters below
## label list below

# GPU memory check
gpu_report()

# display neurons weights as mnist digits
som_grid = plt.figure(figsize=(200, 200)) # width, height in inches
for n in range(map_wth*map_hgt):
    ##
    ## Must make this into time series plot. Shouldnt be the thing
    ##
    #image = weights[n].reshape([28,28]) # x_train[num] is the 784 normalized pixel values
    sub = som_grid.add_subplot(map_wth, map_hgt, n + 1)
    sub.set_axis_off()
    clr = sub.plot(weights[n]) # imshow(image, cmap = plt.get_cmap("jet"), interpolation = "nearest")
    #plt.colorbar(clr)
#plt.savefig("plots/som_weights.png")
plt.show()

#### Mapa 3 por 3 = 9 classes

In [ ]:
# -*- coding: utf-8 -*-

# ####################################################################################################
# GPU-based Self-Organizing-Map by Lyes Khacef.
# Reference: L. Khacef, V. Gripon, and B. Miramond, “GPU-based self-organizing-maps
# for post-labeled few-shot unsupervised learning”, in International Conference On
# Neural Information Processing (ICONIP), 2020.
# ####################################################################################################

# imports
#from imports import *
#from gpu_check import *
#from data_load import *
#from tf_ksom import *
#from label import *
#from test import *

# hyper-parameters
train_data = 60000
label_data = 600
test_data = 10000
input_dim = 400 #784
map_wth = 3
map_hgt = 3
class_nbr = 10
nbr_epochs = 40 # 20
eps_i_list = [1.0]
eps_f_list = [0.01]
eta_i_list = [10.0]
eta_f_list = [0.01]
sigma_kernel = 1.0

# GPU name
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError("GPU device not found!")
print('Found GPU at: {}'.format(device_name))

##
## So, what if theres not test data
##
# load dataset
# x_train, index_train, x_label, index_label, x_test, index_test, label_data = get_dataset(train_data, label_data, test_data)
index_train, x_label, index_label, x_test, index_test, label_data = False, False, False, False, False, False
x_train = npdf1

##
## how the hell
##
# false run_som(false, false, false, eta_f):
def run_som(eps_i, eps_f, eta_i, eta_f):
    print("\nHyper-parameters:   # eps_i = %f   # eps_f = %f   # eta_i = %f   # eta_f = %f" % (eps_i, eps_f, eta_i, eta_f))
    # train the network
    som = KSOM(map_wth, map_hgt, input_dim)
    start_time = timeit.default_timer()
    som.train(nbr_epochs, eps_i, eps_f, eta_i, eta_f, x_train, x_label, index_label, x_test, index_test)
    end_time = timeit.default_timer()
    print("\nSOM training time = ", end_time - start_time)
    weights = som.get_weights().numpy()

    # save the weights
    #np.save("weights/som_weights.npy", weights)

    ##
    ## Now you no label
    ##
    # label the network
    #neuron_label = labeling(label_data, class_nbr, weights, x_label, index_label, sigma_kernel)
    neuron_label = False

    ##
    ## Now you no accuracy
    # test the network
    #accuracy = test(class_nbr, weights, x_test, index_test, neuron_label, sigma_kernel)
    accuracy = False

    return weights, accuracy

# hyper-parameters grid search
hyper_param_list, accuracy_list = [], []
for eps_i in eps_i_list:
    for eps_f in eps_f_list:
        for eta_i in eta_i_list:
            for eta_f in eta_f_list:
                hyper_param_list.append([eps_i, eps_f, eta_i, eta_f])
                weights, accuracy = run_som(eps_i, eps_f, eta_i, eta_f)
                accuracy_list.append(accuracy)


In [ ]:
# display neurons weights as time series
som_grid = plt.figure(figsize=(10, 10)) # width, height in inches
for i in range(map_wth):
  for j in range(map_hgt):
    sub = som_grid.add_subplot(map_wth*map_dpt, map_hgt, 4*k+2*i+j+1)
    for k in range(map_dpt):
        #image = weights[n].reshape([28,28]) # x_train[num] is the 784 normalized pixel values
        sub.set_axis_off()
        clr = sub.plot(range(400),weights[4*k+2*i+j],k)
plt.show()

In [ ]:
print('i:'+str(i)+' and j: '+str(j))

In [ ]:
## hyper parameters below
## label list below

# GPU memory check
gpu_report()

# display neurons weights as mnist digits
som_grid = plt.figure(figsize=(200, 200)) # width, height in inches
for n in range(map_wth*map_hgt*map_dpt):
    ##
    ## Must make this into time series plot. Shouldnt be the thing
    ##
    #image = weights[n].reshape([28,28]) # x_train[num] is the 784 normalized pixel values
    sub = som_grid.add_subplot(map_wth, map_hgt, n + 1)
    sub.set_axis_off()
    clr = sub.plot(weights[n]) # imshow(image, cmap = plt.get_cmap("jet"), interpolation = "nearest")
    #plt.colorbar(clr)
#plt.savefig("plots/som_weights.png")
plt.show()

#### Mapa 3 por 3 por 3 = 27 classes

In [ ]:
# -*- coding: utf-8 -*-

# ####################################################################################################
# GPU-based Self-Organizing-Map by Lyes Khacef.
# Reference: L. Khacef, V. Gripon, and B. Miramond, “GPU-based self-organizing-maps
# for post-labeled few-shot unsupervised learning”, in International Conference On
# Neural Information Processing (ICONIP), 2020.
# ####################################################################################################

# imports
#from imports import *
#from gpu_check import *
#from data_load import *
#from tf_ksom import *
#from label import *
#from test import *

# hyper-parameters
train_data = 60000
label_data = 600
test_data = 10000
input_dim = 400 #784
map_wth = 3
map_hgt = 3
map_dpt = 3
class_nbr = 10
nbr_epochs = 40 # 20
eps_i_list = [1.0]
eps_f_list = [0.01]
eta_i_list = [10.0]
eta_f_list = [0.01]
sigma_kernel = 1.0

# GPU name
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError("GPU device not found!")
print('Found GPU at: {}'.format(device_name))

##
## So, what if theres not test data
##
# load dataset
# x_train, index_train, x_label, index_label, x_test, index_test, label_data = get_dataset(train_data, label_data, test_data)
index_train, x_label, index_label, x_test, index_test, label_data = False, False, False, False, False, False
x_train = npdf1

##
## how the hell
##
# false run_som(false, false, false, eta_f):
def run_som(eps_i, eps_f, eta_i, eta_f):
    print("\nHyper-parameters:   # eps_i = %f   # eps_f = %f   # eta_i = %f   # eta_f = %f" % (eps_i, eps_f, eta_i, eta_f))
    # train the network
    som = KSOM_3D(map_wth, map_hgt, map_dpt, input_dim)
    start_time = timeit.default_timer()
    som.train(nbr_epochs, eps_i, eps_f, eta_i, eta_f, x_train, x_label, index_label, x_test, index_test)
    end_time = timeit.default_timer()
    print("\nSOM training time = ", end_time - start_time)
    weights = som.get_weights().numpy()

    # save the weights
    #np.save("weights/som_weights.npy", weights)

    ##
    ## Now you no label
    ##
    # label the network
    #neuron_label = labeling(label_data, class_nbr, weights, x_label, index_label, sigma_kernel)
    neuron_label = False

    ##
    ## Now you no accuracy
    # test the network
    #accuracy = test(class_nbr, weights, x_test, index_test, neuron_label, sigma_kernel)
    accuracy = False

    return weights, accuracy

# hyper-parameters grid search
hyper_param_list, accuracy_list = [], []
for eps_i in eps_i_list:
    for eps_f in eps_f_list:
        for eta_i in eta_i_list:
            for eta_f in eta_f_list:
                hyper_param_list.append([eps_i, eps_f, eta_i, eta_f])
                weights, accuracy = run_som(eps_i, eps_f, eta_i, eta_f)
                accuracy_list.append(accuracy)


In [ ]:
# display neurons weights as time series
som_grid = plt.figure(figsize=(10, 10)) # width, height in inches
for i in range(map_wth):
  for j in range(map_hgt):
    for k in range(map_dpt):
      sub = som_grid.add_subplot(map_wth, map_hgt, 3*j+i+1)
      #image = weights[n].reshape([28,28]) # x_train[num] is the 784 normalized pixel values
      sub.set_axis_off()
      clr = sub.plot(range(400),weights[9*k+3*j+i],9*k+3*j+i+1)
plt.show()

##### Mapa 2 por 3 = 6 classes

In [ ]:
# -*- coding: utf-8 -*-

# ####################################################################################################
# GPU-based Self-Organizing-Map by Lyes Khacef.
# Reference: L. Khacef, V. Gripon, and B. Miramond, “GPU-based self-organizing-maps
# for post-labeled few-shot unsupervised learning”, in International Conference On
# Neural Information Processing (ICONIP), 2020.
# ####################################################################################################

# imports
#from imports import *
#from gpu_check import *
#from data_load import *
#from tf_ksom import *
#from label import *
#from test import *

# hyper-parameters
train_data = 60000
label_data = 600
test_data = 10000
input_dim = 400 #784
map_wth = 2
map_hgt = 3
class_nbr = 10
nbr_epochs = 40 # 20
eps_i_list = [1.0]
eps_f_list = [0.01]
eta_i_list = [10.0]
eta_f_list = [0.01]
sigma_kernel = 1.0

# GPU name
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError("GPU device not found!")
print('Found GPU at: {}'.format(device_name))

##
## So, what if theres not test data
##
# load dataset
# x_train, index_train, x_label, index_label, x_test, index_test, label_data = get_dataset(train_data, label_data, test_data)
index_train, x_label, index_label, x_test, index_test, label_data = False, False, False, False, False, False
x_train = npdf1

##
## how the hell
##
# false run_som(false, false, false, eta_f):
def run_som(eps_i, eps_f, eta_i, eta_f):
    print("\nHyper-parameters:   # eps_i = %f   # eps_f = %f   # eta_i = %f   # eta_f = %f" % (eps_i, eps_f, eta_i, eta_f))
    # train the network
    som = KSOM(map_wth, map_hgt, input_dim)
    start_time = timeit.default_timer()
    som.train(nbr_epochs, eps_i, eps_f, eta_i, eta_f, x_train, x_label, index_label, x_test, index_test)
    end_time = timeit.default_timer()
    print("\nSOM training time = ", end_time - start_time)
    weights = som.get_weights().numpy()

    # save the weights
    #np.save("weights/som_weights.npy", weights)

    ##
    ## Now you no label
    ##
    # label the network
    #neuron_label = labeling(label_data, class_nbr, weights, x_label, index_label, sigma_kernel)
    neuron_label = False

    ##
    ## Now you no accuracy
    # test the network
    #accuracy = test(class_nbr, weights, x_test, index_test, neuron_label, sigma_kernel)
    accuracy = False

    return weights, accuracy

# hyper-parameters grid search
hyper_param_list, accuracy_list = [], []
for eps_i in eps_i_list:
    for eps_f in eps_f_list:
        for eta_i in eta_i_list:
            for eta_f in eta_f_list:
                hyper_param_list.append([eps_i, eps_f, eta_i, eta_f])
                weights, accuracy = run_som(eps_i, eps_f, eta_i, eta_f)
                accuracy_list.append(accuracy)


In [ ]:
## hyper parameters below
## label list below

# GPU memory check
gpu_report()

# display neurons weights as mnist digits
som_grid = plt.figure(figsize=(200, 200)) # width, height in inches
for n in range(map_wth*map_hgt):
    ##
    ## Must make this into time series plot. Shouldnt be the thing
    ##
    #image = weights[n].reshape([28,28]) # x_train[num] is the 784 normalized pixel values
    sub = som_grid.add_subplot(map_wth, map_hgt, n + 1)
    sub.set_axis_off()
    clr = sub.plot(weights[n]) # imshow(image, cmap = plt.get_cmap("jet"), interpolation = "nearest")
    #plt.colorbar(clr)
#plt.savefig("plots/som_weights.png")
plt.show()

#### Mapa 2 por 2 = 4 classes

In [ ]:
# -*- coding: utf-8 -*-

# ####################################################################################################
# GPU-based Self-Organizing-Map by Lyes Khacef.
# Reference: L. Khacef, V. Gripon, and B. Miramond, “GPU-based self-organizing-maps
# for post-labeled few-shot unsupervised learning”, in International Conference On
# Neural Information Processing (ICONIP), 2020.
# ####################################################################################################

# imports
#from imports import *
#from gpu_check import *
#from data_load import *
#from tf_ksom import *
#from label import *
#from test import *

# hyper-parameters
train_data = 60000
label_data = 600
test_data = 10000
input_dim = 400 #784
map_wth = 2
map_hgt = 2
class_nbr = 10
nbr_epochs = 40 # 20
eps_i_list = [1.0]
eps_f_list = [0.01]
eta_i_list = [10.0]
eta_f_list = [0.01]
sigma_kernel = 1.0

# GPU name
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError("GPU device not found!")
print('Found GPU at: {}'.format(device_name))

##
## So, what if theres not test data
##
# load dataset
# x_train, index_train, x_label, index_label, x_test, index_test, label_data = get_dataset(train_data, label_data, test_data)
index_train, x_label, index_label, x_test, index_test, label_data = False, False, False, False, False, False
x_train = npdf1

##
## make the ref from outside scope
##
som = []

##
## how the hell
##
# false run_som(false, false, false, eta_f):
def run_som(eps_i, eps_f, eta_i, eta_f):
    print("\nHyper-parameters:   # eps_i = %f   # eps_f = %f   # eta_i = %f   # eta_f = %f" % (eps_i, eps_f, eta_i, eta_f))
    # train the network
    som = KSOM(map_wth, map_hgt, input_dim)
    start_time = timeit.default_timer()
    som.train(nbr_epochs, eps_i, eps_f, eta_i, eta_f, x_train, x_label, index_label, x_test, index_test)
    end_time = timeit.default_timer()
    print("\nSOM training time = ", end_time - start_time)
    weights = som.get_weights().numpy()

    # save the weights
    #np.save("weights/som_weights.npy", weights)

    ##
    ## Now you no label
    ##
    # label the network
    #neuron_label = labeling(label_data, class_nbr, weights, x_label, index_label, sigma_kernel)
    neuron_label = False

    ##
    ## Now you no accuracy
    # test the network
    #accuracy = test(class_nbr, weights, x_test, index_test, neuron_label, sigma_kernel)
    accuracy = False

    return weights, accuracy

# hyper-parameters grid search
hyper_param_list, accuracy_list = [], []
for eps_i in eps_i_list:
    for eps_f in eps_f_list:
        for eta_i in eta_i_list:
            for eta_f in eta_f_list:
                hyper_param_list.append([eps_i, eps_f, eta_i, eta_f])
                weights, accuracy = run_som(eps_i, eps_f, eta_i, eta_f)
                accuracy_list.append(accuracy)


In [ ]:
## hyper parameters below
## label list below

# GPU memory check
gpu_report()

# display neurons weights as mnist digits
som_grid = plt.figure(figsize=(10, 10)) # width, height in inches
for n in range(map_wth*map_hgt):
    ##
    ## Must make this into time series plot. Shouldnt be the thing
    ##
    #image = weights[n].reshape([28,28]) # x_train[num] is the 784 normalized pixel values
    sub = som_grid.add_subplot(map_wth, map_hgt, n + 1)
    sub.set_axis_off()
    clr = sub.plot(weights[n]) # imshow(image, cmap = plt.get_cmap("jet"), interpolation = "nearest")
    #plt.colorbar(clr)
#plt.savefig("plots/som_weights.png")
#plt.show()

## Mapa 2 por 2 por 2 = 8 classes

In [ ]:
# -*- coding: utf-8 -*-

# ####################################################################################################
# GPU-based Self-Organizing-Map by Lyes Khacef.
# Reference: L. Khacef, V. Gripon, and B. Miramond, “GPU-based self-organizing-maps
# for post-labeled few-shot unsupervised learning”, in International Conference On
# Neural Information Processing (ICONIP), 2020.
# ####################################################################################################

# imports
#from imports import *
#from gpu_check import *
#from data_load import *
#from tf_ksom import *
#from label import *
#from test import *

# hyper-parameters
train_data = 60000
label_data = 600
test_data = 10000
input_dim = 400 #784
map_wth = 2
map_hgt = 2
map_dpt = 2
class_nbr = 10
nbr_epochs = 40 # 20
eps_i_list = [1.0]
eps_f_list = [0.01]
eta_i_list = [10.0]
eta_f_list = [0.01]
sigma_kernel = 1.0

# GPU name
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError("GPU device not found!")
print('Found GPU at: {}'.format(device_name))

##
## So, what if theres not test data
##
# load dataset
# x_train, index_train, x_label, index_label, x_test, index_test, label_data = get_dataset(train_data, label_data, test_data)
index_train, x_label, index_label, x_test, index_test, label_data = False, False, False, False, False, False
x_train = npdf1

##
## make the ref from outside scope
##
som = []

##
## how the hell
##
# false run_som(false, false, false, eta_f):
def run_som(eps_i, eps_f, eta_i, eta_f):
    print("\nHyper-parameters:   # eps_i = %f   # eps_f = %f   # eta_i = %f   # eta_f = %f" % (eps_i, eps_f, eta_i, eta_f))
    # train the network
    som = KSOM_3D(map_wth, map_hgt, map_dpt, input_dim)
    start_time = timeit.default_timer()
    som.train(nbr_epochs, eps_i, eps_f, eta_i, eta_f, x_train, x_label, index_label, x_test, index_test)
    end_time = timeit.default_timer()
    print("\nSOM training time = ", end_time - start_time)
    weights = som.get_weights().numpy()

    # save the weights
    #np.save("weights/som_weights.npy", weights)

    ##
    ## Now you no label
    ##
    # label the network
    #neuron_label = labeling(label_data, class_nbr, weights, x_label, index_label, sigma_kernel)
    neuron_label = False

    ##
    ## Now you no accuracy
    # test the network
    #accuracy = test(class_nbr, weights, x_test, index_test, neuron_label, sigma_kernel)
    accuracy = False

    return weights, accuracy

# hyper-parameters grid search
hyper_param_list, accuracy_list = [], []
for eps_i in eps_i_list:
    for eps_f in eps_f_list:
        for eta_i in eta_i_list:
            for eta_f in eta_f_list:
                hyper_param_list.append([eps_i, eps_f, eta_i, eta_f])
                weights, accuracy = run_som(eps_i, eps_f, eta_i, eta_f)
                accuracy_list.append(accuracy)


In [ ]:
# display neurons weights as time series
som_grid = plt.figure(figsize=(10, 10)) # width, height in inches
for i in range(map_wth):
  for j in range(map_hgt):
    for k in range(map_dpt):
      sub = som_grid.add_subplot(map_wth, map_hgt, 2*j+i+1)
      #image = weights[n].reshape([28,28]) # x_train[num] is the 784 normalized pixel values
      sub.set_axis_off()
      clr = sub.plot(range(400),weights[4*k+2*i+j],k)
plt.show()

Certo... analisando rapidamente...

<table>
  <tr>
    <td>
      1,1, azul eh 0-ply e verde 90-ply
    </td>
    <td>
      1,2, azul eh 0-ply e verde 90-ply
    </td>
  </tr>

  <tr>
    <td>
      2,1, azul eh +-45-ply e verde 0-ply<br>
      ou<br>
      2,1, azul eh 90-ply invertido e verrde eh +-45-ply
    </td>
    <td>
      2,2, azul eh 0-ply e verde +-45-ply
    </td>
  </tr>
</table>

In [ ]:
# display neurons weights as time series
som_grid = plt.figure(figsize=(10, 10)) # width, height in inches
for i in range(map_wth):
  for j in range(map_hgt):
    sub = som_grid.add_subplot(map_wth*map_dpt, map_hgt, 4*k+2*i+j+1,projection='3d')
    for k in range(map_dpt):
        #image = weights[n].reshape([28,28]) # x_train[num] is the 784 normalized pixel values
        sub.set_axis_off()
        clr = sub.plot(range(400),weights[4*k+2*i+j],k)
plt.show()

In [ ]:
weights.shape

# 4. Unified Distance Matrix

In [ ]:
plt.imshow(umat, cmap=plt.cm.get_cmap('RdYlBu_r'), alpha=1)

In [ ]:
## hyper parameters below
## label list below

# GPU memory check
gpu_report()

# display neurons weights as mnist digits
som_grid = plt.figure(figsize=(10, 10)) # width, height in inches
for n in range(map_wth*map_hgt):
    ##
    ## Must make this into time series plot. Shouldnt be the thing
    ##
    #image = weights[n].reshape([28,28]) # x_train[num] is the 784 normalized pixel values
    sub = som_grid.add_subplot(map_wth, map_hgt, n + 1)
    sub.set_axis_off()
    clr = sub.plot(weights[n]) # imshow(image, cmap = plt.get_cmap("jet"), interpolation = "nearest")
    #plt.colorbar(clr)
#plt.savefig("plots/som_weights.png")
plt.show()

In [ ]:
plt.savefig("plots/som_weights.png")

In [ ]:

# best hyper-parameters
best_accuracy = np.max(accuracy_list)
best_hyper_param = hyper_param_list[np.argmax(accuracy_list)]
print("Best accuracy = ", best_accuracy)
print("Best hyper-parameters:   # eps_i = %f   # eps_f = %f   # sig_i = %f   # sig_f = %f" % (best_hyper_param[0], best_hyper_param[1], best_hyper_param[2], best_hyper_param[3]))


In [ ]:
label_list = [label_data]
for label_data in label_list:
    print("\n---------- Labels = %d ----------" % label_data)
    run_labeling(weights, label_data, x_train, index_train, x_test, index_test)